<a href="https://colab.research.google.com/github/eyaler/workshop/blob/master/Hebrew_bigrams_and_slurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install umap-learn
!pip install hdbscan
!wget --no-check-certificate -nc https://drive.google.com/uc?id=1xZbnVEFVSS1zSQ6a9FH1-_GgDM3AH3wr -O mwevectors
!wget --no-check-certificate -nc https://drive.google.com/uc?id=1OVMKK5RPhRPm5GxGunvFytnJktkc6yr9 -O mwevectors.vectors.npy
!wget --no-check-certificate -nc https://drive.google.com/uc?id=1DRk-ziMGop5fIYdnq2yk6IBJOughW9_W -O mwevectors_filter_prefix
!wget --no-check-certificate -nc https://drive.google.com/uc?id=1xP74-90nOfTjlABs3ntsWcfCt6syZEMJ -O mwevectors_filter_prefix.vectors.npy

In [ ]:
from gensim.models import KeyedVectors
wv_model_with_prefix = KeyedVectors.load('mwevectors')
wv_model = KeyedVectors.load('mwevectors_filter_prefix')
print(len(wv_model_with_prefix.index2word))
print(len(wv_model.index2word))

In [ ]:
print(wv_model.index2word[:15])

In [ ]:
import sklearn.cluster as cluster_algo
import numpy as np

def cluster_save(wv_model, n_clusters, limit=None):
    words = wv_model.index2word
    vectors = wv_model.vectors[:limit]

    labels = cluster_algo.KMeans(n_clusters=n_clusters).fit_predict(vectors)
    with open('clusters.txt', 'w', encoding='utf-8') as f:
        for cluster in range(max(labels)+1):
            if cluster in labels:
                s = str(cluster) + ': '
                for index in np.argwhere(labels == cluster):
                    s += words[int(index)] + ', '
                f.write(s+'\n')
                print(s)
    return labels

In [ ]:
clusters = cluster_save(wv_model, 10, limit=100)

In [ ]:
import umap
from matplotlib import pyplot as plt
import matplotlib.cm as cm

def umap_plot(wv_model, clusters=None, limit=None):
    words = wv_model.index2word
    vectors = wv_model.vectors[:limit]

    umap_model = umap.UMAP(n_neighbors=15, random_state=42, metric='cosine')
    low_dim = umap_model.fit_transform(vectors)
    
    plt.figure(figsize=(18, 18)) 
    for i, xy in enumerate(low_dim):
        cmap = None
        if clusters is not None and i<len(clusters):
          cmap = cm.jet(clusters[i]/len(set(clusters[:limit])))
        plt.scatter(*xy, color=cmap)
        plt.annotate(words[i][::-1],
                     xy=xy,
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
    return low_dim
    

In [ ]:
low_dim = umap_plot(wv_model, clusters, limit=100)

###קטע מתוך השיר
#"אלחושך הפריעיה"
###מאת איל גרוס
###מסרים שקיבלתי מצבא האריות של הצל
###(בשינוי סדר)
---
#לא רק ל-יפרצופי זין
#ול-ישמאלנים זין
#ול-ישרמוטות שמאלנים
#ול-יבני זונות שמאלנים מסריחים
#ול-ימחבלים מסריחים
#ול-ישרמוטות שמאלנים מסריחים וטיפשים
#ול-יחתיכות דפוקים טיפשים
#ול-יחתיכות קוקסינלים
#ול-יחתיכות טמבלים
#ול-יטמבלים אוהבי ערבים


In [ ]:
bad_words = ['דפוק','טיפש','מסריח','זין']
bad_bigrams = [w for w in wv_model.index2word if any(b in w for b in bad_words)]
print(len(bad_bigrams))
print(bad_bigrams)

In [ ]:
bad_words = ['זונה','זונות','שרמוט','מזדיי','טמבל']
bad_bigrams = [w for w in wv_model.index2word if any(b in w for b in bad_words)]
print(len(bad_bigrams))
print(bad_bigrams)

In [ ]:
vw_model.most_similar(bad_bigrams[0],topn=10)

In [ ]:
# using shared neighbors to get higher precision 
# https://imada.sdu.dk/~zimek/publications/SSDBM2010/SNN-SSDBM2010-preprint.pdf
from collections import Counter
more_bad = bad_bigrams.copy()
for bad in bad_bigrams:
  more_bad += [n[0] for n in vw_model.most_similar(bad, topn=10)]
bad_candidates = Counter(more_bad)
bad_candidates.most_common()

In [ ]:
new_bad = set([k for k,v in bad_candidates.items() if v>=4]+bad_bigrams)
print(len(new_bad))
print(new_bad)

In [ ]:
print(new_bad-set(bad_bigrams))

In [ ]:
more_bad2 = [b for b in more_bad if b in new_bad]
for bad in new_bad:
  more_bad2 += [n[0] for n in vw_model.most_similar(bad, topn=10)]
bad_candidates2 = Counter(more_bad2)
bad_candidates2.most_common()

In [ ]:
new_bad2 = set([k for k,v in bad_candidates2.items() if v>=4])|new_bad
print(len(new_bad2))
print(new_bad2)

In [ ]:
print(new_bad2-new_bad)